# Cleaning the Data

In [2]:
import pandas as pd

In [3]:
# Load the dataset
file_path = r"C:\Users\info\Desktop\project3team11\Resources\U.S._Chronic_Disease_Indicators.csv"
data = pd.read_csv(file_path)

In [4]:
data.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,TopicID,QuestionID,ResponseID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
0,2019,2019,AR,Arkansas,BRFSS,Diabetes,Diabetes among adults,NaN,%,Crude Prevalence,...,DIA,DIA01,NaN,CRDPREV,SEX,SEXM,NaN,NaN,NaN,NaN
1,2019,2019,ID,Idaho,BRFSS,Diabetes,Diabetes among adults,NaN,%,Crude Prevalence,...,DIA,DIA01,NaN,CRDPREV,SEX,SEXM,NaN,NaN,NaN,NaN
2,2019,2019,IN,Indiana,YRBSS,Sleep,Short sleep duration among high school students,NaN,%,Crude Prevalence,...,SLEP,SLP02,NaN,CRDPREV,GRADE,GRD12,NaN,NaN,NaN,NaN
3,2019,2019,IA,Iowa,NVSS,Asthma,"Asthma mortality among all people, underlying ...",NaN,Number,Number,...,AST,AST01,NaN,NMBR,OVERALL,OVR,NaN,NaN,NaN,NaN
4,2019,2019,IA,Iowa,BRFSS,Asthma,Current asthma among adults,NaN,%,Crude Prevalence,...,AST,AST02,NaN,CRDPREV,AGE,AGE1844,NaN,NaN,NaN,NaN


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309215 entries, 0 to 309214
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   YearStart                  309215 non-null  int64  
 1   YearEnd                    309215 non-null  int64  
 2   LocationAbbr               309215 non-null  object 
 3   LocationDesc               309215 non-null  object 
 4   DataSource                 309215 non-null  object 
 5   Topic                      309215 non-null  object 
 6   Question                   309215 non-null  object 
 7   Response                   0 non-null       float64
 8   DataValueUnit              309215 non-null  object 
 9   DataValueType              309215 non-null  object 
 10  DataValue                  209196 non-null  float64
 11  DataValueAlt               209196 non-null  float64
 12  DataValueFootnoteSymbol    101716 non-null  object 
 13  DataValueFootnote          10

In [6]:
# Drop columns with 0 non-null values
# Drop only the columns that exist in the dataset
columns_to_drop = [
    'Response', 'StratificationCategory2', 'Stratification2', 'StratificationCategory3', 'Stratification3', 
    'ResponseID', 'StratificationCategoryID2', 'StratificationID2','StratificationCategoryID3', 
    'StratificationID3', 'YearStart', 
    'DataValueFootnoteSymbol', 'DataValueFootnote', 'StratificationID1', 'StratificationCategoryID1'
    
]

# Filter columns_to_drop to include only existing columns
columns_to_drop = [col for col in columns_to_drop if col in data.columns]

# Drop the filtered list of columns
data.drop(columns=columns_to_drop, inplace=True)

print("Dropped unnecessary columns successfully!")


Dropped unnecessary columns successfully!


In [7]:
# Fill missing 'Geolocation' with 'Unknown'
data['Geolocation'].fillna('Unknown', inplace=True)

C:\Users\info\AppData\Local\Temp\ipykernel_20964\2711780504.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Geolocation'].fillna('Unknown', inplace=True)


In [8]:
# Extract longitude and latitude into separate columns
data[['Longitude', 'Latitude']] = data['Geolocation'].str.extract(r'POINT \(([-\d.]+) ([-\d.]+)\)')
#Rename Columns
data.rename(columns={'YearEnd': 'Year'}, inplace=True)
data.rename(columns={'LocationDesc': 'Location'}, inplace=True)
data.rename(columns={'LocationAbbr': 'LocationID'}, inplace=True)
data.rename(columns={'DataValue': 'Value'}, inplace=True)

In [9]:
# Drop duplicate rows
data = data.drop_duplicates()

In [10]:
# List of stratification categories
StratificationCategory1 = ['Sex', 'Age', 'Race/Ethnicity', 'Grade', 'Overall']

# Pivot the data: Create separate columns for each StratificationCategory
for category in StratificationCategory1:
    # Assign values from Stratification1 where StratificationCategory1 matches the current category
    data[category] = data['Stratification1'].where(data['StratificationCategory1'] == category)

# Drop the original columns if no longer needed
data.drop(columns=['StratificationCategory1', 'Stratification1'], inplace=True)

# Fill missing values in the new columns with 'Unknown' (optional)
data[StratificationCategory1] = data[StratificationCategory1].fillna('Unknown')

In [52]:
#Drop 'Overall' Column
data.drop(columns=[ 'Overall','DataValueAlt'], inplace=True)

In [14]:
print(data.columns)

Index(['Year', 'Sex', 'Age', 'Grade', 'Race/Ethnicity', 'DataSource',
       'Location', 'LocationID', 'LocationID', 'Geolocation', 'Longitude',
       'Latitude', 'Topic', 'TopicID', 'Question', 'QuestionID', 'Value',
       'DataValueUnit', 'DataValueType', 'DataValueTypeID',
       'LowConfidenceLimit', 'HighConfidenceLimit'],
      dtype='object')


In [15]:
# Define the new column order
new_column_order = ['Year','Sex', 'Age', 'Grade', 'Race/Ethnicity', 'DataSource',
                    'Location','LocationID','Geolocation', 'Longitude', 'Latitude', 
                    'Topic', 'TopicID', 'Question', 'QuestionID', 'Value', 'DataValueUnit',
                    'DataValueType', 'DataValueTypeID', 'LowConfidenceLimit', 'HighConfidenceLimit' ]
# Reorder the DataFrame
data_reordered = data[new_column_order]

# Columns sorted by data source type
data = data_reordered.sort_values(by=['DataSource'], ascending=[True])
data.reset_index(drop=True, inplace=True)

In [16]:
# Review the final data set
print(data)

        Year      Sex      Age     Grade             Race/Ethnicity  \
0       2019  Unknown  Unknown   Unknown                   Hispanic   
1       2021  Unknown  Unknown   Unknown  Multiracial, non-Hispanic   
2       2021     Male  Unknown   Unknown                    Unknown   
3       2019  Unknown  Unknown   Unknown                    Unknown   
4       2019  Unknown  Unknown   Unknown                    Unknown   
...      ...      ...      ...       ...                        ...   
309210  2021  Unknown  Unknown   Unknown                   Hispanic   
309211  2021  Unknown  Unknown   Unknown        Asian, non-Hispanic   
309212  2021  Unknown  Unknown  Grade 11                    Unknown   
309213  2021  Unknown  Unknown  Grade 10                    Unknown   
309214  2019  Unknown  Unknown   Unknown                    Unknown   

       DataSource        Location  LocationID  LocationID  \
0             ACS      California          CA           6   
1             ACS        

In [17]:
# Save the dataset as a CSV file
file_path = r'C:\Users\info\Desktop\project3team11\Resources\final2_cleaned_dataset.csv'
data.to_csv(file_path, index=False)  
